In [1]:
import os
import opensim as osim
import numpy as np
import json

In [2]:
path_comak_model = '../../../models/knee_healthy/lenhart2015/lenhart2015.osim'
comak_model = osim.Model(path_comak_model)

[info] Loaded model lenhart2015 from file ../../../models/knee_healthy/lenhart2015/lenhart2015.osim


In [3]:
dict_bodies = {}

comak_bodies = comak_model.getBodySet()

for body_idx in range(comak_bodies.getSize()):
    body = comak_bodies.get(body_idx)
    name = body.getName()
    attached_geo = body.getPropertyByName('attached_geometry')
    n_geos = attached_geo.size()
    geos = np.zeros((n_geos, 3))
    for i in range(n_geos):
        geos[i, :] = [body.get_attached_geometry(i).get_scale_factors()[x] for x in range(3)]
    print(name)
    
    dict_bodies[name] = {
        'mean_scale': np.mean(geos, axis=0),
        'std_scale': np.std(geos, axis=0),
        'const_axis_scale': np.std(geos, axis=0) < 1e-4,
        'const_scale': np.std(geos) < 1e-4
    }
    

pelvis
femur_r
femur_distal_r
patella_r
tibia_proximal_r
tibia_r
talus_r
calcn_r
toes_r
femur_l
patella_l
tibia_l
talus_l
calcn_l
toes_l
torso
neckhead
humerus_r
ulna_r
radius_r
hand_r
humerus_l
ulna_l
radius_l
hand_l


/miniconda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:251: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/miniconda/lib/python3.7/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
femur_r_lenhart_translation = np.asarray([-0.0055513564376633642, -0.37418143637169787, -0.0011706232813375212])
# offset = np.asarray([-0.00808954, -0.0035348, -0.00148474])
# femur_r_lenhart_translation += offset
femur_r_walker_location_in_parent = np.asarray([-0.0044999999999999997, -0.40960000000000002, -0.00175])

femur_r_lenhart_translation / femur_r_walker_location_in_parent


array([1.23363476, 0.9135289 , 0.66892759])

In [5]:
ankle_tibia_offset_rajagopal = np.asarray([-0.01, -0.40000000000000002, 0])
ankle_tibia_offset_lenhart = np.asarray([-0.010082668143607483, -0.40330672574429932, 0])

ankle_tibia_offset_lenhart / ankle_tibia_offset_rajagopal


/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


array([1.00826681, 1.00826681,        nan])

In [6]:
femur_r_scale = 0.9135289
tibia_r_scale = 1.00826681

In [7]:
dict_bodies

{'pelvis': {'mean_scale': array([1.19585345, 1.06053591, 1.19585345]),
  'std_scale': array([0., 0., 0.]),
  'const_axis_scale': array([ True,  True,  True]),
  'const_scale': False},
 'femur_r': {'mean_scale': array([nan, nan, nan]),
  'std_scale': array([nan, nan, nan]),
  'const_axis_scale': array([False, False, False]),
  'const_scale': False},
 'femur_distal_r': {'mean_scale': array([1., 1., 1.]),
  'std_scale': array([0., 0., 0.]),
  'const_axis_scale': array([ True,  True,  True]),
  'const_scale': True},
 'patella_r': {'mean_scale': array([1., 1., 1.]),
  'std_scale': array([0., 0., 0.]),
  'const_axis_scale': array([ True,  True,  True]),
  'const_scale': True},
 'tibia_proximal_r': {'mean_scale': array([1., 1., 1.]),
  'std_scale': array([0., 0., 0.]),
  'const_axis_scale': array([ True,  True,  True]),
  'const_scale': True},
 'tibia_r': {'mean_scale': array([nan, nan, nan]),
  'std_scale': array([nan, nan, nan]),
  'const_axis_scale': array([False, False, False]),
  'const_

## Notes:
The above shows that the only bodies that arent consistently scaled on all axes are: 
   - Pelvi
   - Torso
   
The only ones without scales are tibia_r and femur_r because they have no geometry anymore. 

- Tibia scaling is consistent on all axes -> so will just use that. 
- Femur scaling is less clear, but becuase large errors potentially in med/lat and ant/post measurements, will just use long-axis length as scaling factor

Redoing the above and only saving the scaling factors along each axis

In [8]:
dict_body_scales = {}

comak_bodies = comak_model.getBodySet()

for body_idx in range(comak_bodies.getSize()):
    body = comak_bodies.get(body_idx)
    name = body.getName()
    attached_geo = body.getPropertyByName('attached_geometry')
    n_geos = attached_geo.size()
    geos = np.zeros((n_geos, 3))
    for i in range(n_geos):
        geos[i, :] = [body.get_attached_geometry(i).get_scale_factors()[x] for x in range(3)]
    print(name)
    
#     geo = body.get_attached_geometry(0)
#     scale = [geo.get_scale_factors()[x] for x in range(3)]
    if name == 'femur_r':
        scale = [0.9135289, 0.9135289, 0.9135289]
    elif name == 'tibia_r':
        scale = [1.00826681, 1.00826681, 1.00826681]
    else:
        scale = geos[0,:].tolist()
    dict_body_scales[name] = scale 

pelvis
femur_r
femur_distal_r
patella_r
tibia_proximal_r
tibia_r
talus_r
calcn_r
toes_r
femur_l
patella_l
tibia_l
talus_l
calcn_l
toes_l
torso
neckhead
humerus_r
ulna_r
radius_r
hand_r
humerus_l
ulna_l
radius_l
hand_l


In [9]:
with open('./body_scaling.json', 'w') as f:
    json.dump(dict_body_scales, f, indent=4)